# 诗歌生成

# 数据处理

In [1]:
import numpy as np
import tensorflow as tf
import collections
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import layers, optimizers, datasets

start_token = 'bos'
end_token = 'eos'

def process_dataset(fileName):
    examples = []
    with open(fileName, 'r', encoding='utf-8') as fd:
        for line in fd:
            outs = line.strip().split(':')
            content = ''.join(outs[1:])
            ins = [start_token] + list(content) + [end_token] 
            if len(ins) > 200:
                continue
            examples.append(ins)
            
    counter = collections.Counter()
    for e in examples:
        for w in e:
            counter[w]+=1
    
    sorted_counter = sorted(counter.items(), key=lambda x: -x[1])  # 排序
    words, _ = zip(*sorted_counter)
    words = ('PAD', 'UNK') + words[:len(words)]
    word2id = dict(zip(words, range(len(words))))
    id2word = {word2id[k]:k for k in word2id}
    
    indexed_examples = [[word2id[w] for w in poem]
                        for poem in examples]
    seqlen = [len(e) for e in indexed_examples]
    
    instances = list(zip(indexed_examples, seqlen))
    
    return instances, word2id, id2word

def poem_dataset():
    instances, word2id, id2word = process_dataset('./poems.txt')
    ds = tf.data.Dataset.from_generator(
        lambda: [ins for ins in instances], 
        (tf.int64, tf.int64), (tf.TensorShape([None]),
        tf.TensorShape([]))
    )
    ds = ds.shuffle(buffer_size=10240)
    ds = ds.padded_batch(100, padded_shapes=(tf.TensorShape([None]),tf.TensorShape([])))
    ds = ds.map(lambda x, seqlen: (x[:, :-1], x[:, 1:], seqlen-1))
    return ds, word2id, id2word

2024-11-27 12:46:41.745390: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0


# 模型代码， 完成建模代码

In [2]:
class myRNNModel(keras.Model):
    def __init__(self, w2id):
        super(myRNNModel, self).__init__()
        self.v_sz = len(w2id)
        self.embed_layer = tf.keras.layers.Embedding(
            self.v_sz, 64, batch_input_shape=[None, None]
        )
        self.rnncell = tf.keras.layers.SimpleRNNCell(128)
        self.rnn_layer = tf.keras.layers.RNN(self.rnncell, return_sequences=True)
        self.dense = tf.keras.layers.Dense(self.v_sz)
    
    @tf.function
    def call(self, inp_ids):
        # print(self.v_sz, inp_ids.shape)
        # 嵌入层
        inp_emb = self.embed_layer(inp_ids)  # (batch_size, timesteps, embed_dim)
        # RNN 层
        rnn_output = self.rnn_layer(inp_emb)  # (batch_size, timesteps, rnn_units)
        # 全连接层
        logits = self.dense(rnn_output)  # (batch_size, timesteps, vocab_size)
        # print(type(logits))
        return logits
    
    @tf.function
    def get_next_token(self, x, state):
        '''
        shape(x) = [b_sz,] 
        '''
    
        inp_emb = self.embed_layer(x) #shape(b_sz, emb_sz)
        h, state = self.rnncell.call(inp_emb, state) # shape(b_sz, h_sz)
        logits = self.dense(h) # shape(b_sz, v_sz)
        out = tf.argmax(logits, axis=-1)
        return out, state

## 一个计算sequence loss的辅助函数，只需了解用途。

In [3]:
def mkMask(input_tensor, maxLen):
    shape_of_input = tf.shape(input_tensor)
    shape_of_output = tf.concat(axis=0, values=[shape_of_input, [maxLen]])

    oneDtensor = tf.reshape(input_tensor, shape=(-1,))
    flat_mask = tf.sequence_mask(oneDtensor, maxlen=maxLen)
    return tf.reshape(flat_mask, shape_of_output)


def reduce_avg(reduce_target, lengths, dim):
    """
    Args:
        reduce_target : shape(d_0, d_1,..,d_dim, .., d_k)
        lengths : shape(d0, .., d_(dim-1))
        dim : which dimension to average, should be a python number
    """
    shape_of_lengths = lengths.get_shape()
    shape_of_target = reduce_target.get_shape()
    if len(shape_of_lengths) != dim:
        raise ValueError(('Second input tensor should be rank %d, ' +
                         'while it got rank %d') % (dim, len(shape_of_lengths)))
    if len(shape_of_target) < dim+1 :
        raise ValueError(('First input tensor should be at least rank %d, ' +
                         'while it got rank %d') % (dim+1, len(shape_of_target)))

    rank_diff = len(shape_of_target) - len(shape_of_lengths) - 1
    mxlen = tf.shape(reduce_target)[dim]
    mask = mkMask(lengths, mxlen)
    if rank_diff!=0:
        len_shape = tf.concat(axis=0, values=[tf.shape(lengths), [1]*rank_diff])
        mask_shape = tf.concat(axis=0, values=[tf.shape(mask), [1]*rank_diff])
    else:
        len_shape = tf.shape(lengths)
        mask_shape = tf.shape(mask)
    lengths_reshape = tf.reshape(lengths, shape=len_shape)
    mask = tf.reshape(mask, shape=mask_shape)

    mask_target = reduce_target * tf.cast(mask, dtype=reduce_target.dtype)

    red_sum = tf.reduce_sum(mask_target, axis=[dim], keepdims=False)
    red_avg = red_sum / (tf.cast(lengths_reshape, dtype=tf.float32) + 1e-30)
    return red_avg

# 定义loss函数，定义训练函数

In [4]:
@tf.function
def compute_loss(logits, labels, seqlen):
    losses = tf.nn.sparse_softmax_cross_entropy_with_logits(
            logits=logits, labels=labels)
    losses = reduce_avg(losses, seqlen, dim=1)
    return tf.reduce_mean(losses)
    
@tf.function
def train_one_step(model, optimizer, x, y, seqlen):
    '''
    完成一步优化过程，可以参考之前做过的模型 
    '''
    with tf.GradientTape() as tape:
        # 前向传播
        logits = model(x)  # shape: (batch_size, sequence_length, vocab_size)
        
        # 计算损失
        loss = compute_loss(logits, y, seqlen)
    
    # 计算梯度
    gradients = tape.gradient(loss, model.trainable_variables)
    
    # 更新权重
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    
    return loss

def train(epoch, model, optimizer, ds):
    loss = 0.0
    accuracy = 0.0
    for step, (x, y, seqlen) in enumerate(ds):
        loss = train_one_step(model, optimizer, x, y, seqlen)

        if step % 50 == 0:
            print('epoch', epoch, ': loss', loss.numpy())

    return loss

# 训练优化过程

In [5]:
optimizer = optimizers.Adam(0.0005)
train_ds, word2id, id2word = poem_dataset()
model = myRNNModel(word2id)

for epoch in range(10):
    loss = train(epoch, model, optimizer, train_ds)

2024-11-27 12:46:44.888645: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcuda.so.1
2024-11-27 12:46:44.934655: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1733] Found device 0 with properties: 
pciBusID: 0000:88:00.0 name: NVIDIA GeForce RTX 3080 computeCapability: 8.6
coreClock: 1.71GHz coreCount: 68 deviceMemorySize: 9.77GiB deviceMemoryBandwidth: 707.88GiB/s
2024-11-27 12:46:44.935014: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1733] Found device 1 with properties: 
pciBusID: 0000:89:00.0 name: NVIDIA GeForce RTX 3080 computeCapability: 8.6
coreClock: 1.71GHz coreCount: 68 deviceMemorySize: 9.77GiB deviceMemoryBandwidth: 707.88GiB/s
2024-11-27 12:46:44.935041: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
2024-11-27 12:46:44.941113: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcublas

epoch 0 : loss 8.819988
epoch 0 : loss 6.7392507
epoch 0 : loss 6.5729027
epoch 0 : loss 6.5284057
epoch 0 : loss 6.520408
epoch 0 : loss 6.5873785
epoch 0 : loss 6.492665
epoch 0 : loss 6.574579
epoch 0 : loss 6.530363
epoch 1 : loss 6.6035676
epoch 1 : loss 6.4748363
epoch 1 : loss 6.4844017
epoch 1 : loss 6.412931
epoch 1 : loss 6.4022956
epoch 1 : loss 6.336051
epoch 1 : loss 6.2545834
epoch 1 : loss 6.1453314
epoch 1 : loss 6.122844
epoch 2 : loss 6.254203
epoch 2 : loss 5.9119473
epoch 2 : loss 5.901873
epoch 2 : loss 5.8921022
epoch 2 : loss 5.8151
epoch 2 : loss 5.921626
epoch 2 : loss 5.9104853
epoch 2 : loss 5.903154
epoch 2 : loss 5.868508
epoch 3 : loss 5.9410768
epoch 3 : loss 5.7757573
epoch 3 : loss 5.738744
epoch 3 : loss 5.7591186
epoch 3 : loss 5.749307
epoch 3 : loss 5.740973
epoch 3 : loss 5.735068
epoch 3 : loss 5.7854915
epoch 3 : loss 5.738645
epoch 4 : loss 5.778041
epoch 4 : loss 5.695814
epoch 4 : loss 5.6809554
epoch 4 : loss 5.6814985
epoch 4 : loss 5.682462

# 生成过程

In [6]:
def gen_sentence():
    state = [tf.random.normal(shape=(1, 128), stddev=0.5), tf.random.normal(shape=(1, 128), stddev=0.5)]
    cur_token = tf.constant([word2id['bos']], dtype=tf.int32)
    collect = [] # 保存每一步生成汉字对应于字典中的整数编码
    '''
    填空三 建立for循环，连续调用model.get_next_token函数，生成长度为50的诗句
    '''
    for _ in range(50):  # 生成50个token
        next_token, state = model.get_next_token(cur_token, state)  # 获取下一个token，并更新状态
        collect.append(next_token.numpy()[0])  # 将生成的token添加到collect列表中
        cur_token = next_token  # 当前token为生成的下一个token

    return [id2word[t] for t in collect]
print(''.join(gen_sentence()))

水上春风起，风风落月中。eos来无处处，不见此人心。eos有无人事，何人不可知。eos来无处处，不见此人心。eos有
